# Using Retrieval-Augmented Generation to Search a Movie Database

Retrieval-augmented generation, or "RAG", is a technique used with large language models to provide additional context without fine-tuning or retraining. It enhances the ability of language models to provide factual responses, which is a limitation of classical setups.

The goal of this project is to build a question-answering bot for movie-related questions. To achieve this, we will use RAG to provide factual information to the language model. We will upload movie descriptions to a vector database and use it to search for relevant context for the language model.

We will be using the following tools and models:
- [OpenAI](https://openai.com)'s `gpt-3.5-turbo` model for prompt completions
- OpenAI's `text-embedding-ada-002` model to create vector embeddings
- [Pinecone](https://www.pinecone.io/) as the vector database to store the embeddings
- [langchain](https://www.langchain.com/) as the tool to interact with OpenAI and Pinecone

The dataset used for this project is sourced from the Kaggle dataset [IMDb Movies/Shows with Descriptions](https://www.kaggle.com/datasets/ishikajohari/imdb-data-with-descriptions).

## Before you begin

To get started with this project, you'll need a developer account for OpenAI and Pinecone. Follow the steps in the [getting-started.ipynb](https://app.datacamp.com/workspace/w/f1d996aa-0aaa-47e3-bd61-2b5b5a0fa558/edit/getting-started.ipynb) notebook to create an API key and store it in Workspace.

For this project, we will assume that you have already set the `OPENAI_API_KEY` and `PINECONE_API_KEY` environment variables.

In [27]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONCE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONCE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")

## Task 0: Setup

To perform this analysis, we need to install the following packages:

- `openai`: for interacting with OpenAI
- `pinecone-client`: for interacting with Pinecone
- `tiktoken`: a string encoder that generates tokens used by OpenAI. It is useful for estimating the number of tokens used.
- `langchain`: the toolchain used to interact with OpenAI and Pinecone

### Instructions

Run the cell below to install the corresponding packages.

In [2]:
# Below we installed specific versions of the packages
# Feel free to experiment with different versions
# However, the workspace below is only tested with these specific versions
# !pip install pinecone-client==2.2.2 openai==0.28.0 tiktoken==0.5.1 langchain==0.0.291

## Task 1: Import the Movies Data

We'll start with importing the dataset we mentioned at the top of this project. You have the dataset available as a CSV in your workspace: `"IMDB.csv"`. We need to import the dataset and transform it into a convenient format.

### Instructions

- Import the `pandas` package as `pd`
- Import `"IMDB.csv"` into a variable `movies` and do the following transformation on `movies`:
  - Rename `primaryTitle` to `movie_title` and `Description` to `movie_description`
  - Create a column `source` that contains the identifier of the movie, prefixed by `"https://www.imdb.com/title/"`. The end result should be a working link to the movie. The identifier can be found in the `"tconst"` column in `"IMDB.csv"`. For example, `"https://www.imdb.com/title/tt0102926/"`.
  - Filter out all rows that do not have `"movie"` as a `titleType`
  - Select the `movie_title`, `movie_description`, `source` and `genres` columns
- Show the head of `movies`

In [3]:
# Import pandas as pd
import pandas as pd

#Import display and markdown to get better-looking outputs
from IPython.display import display, Markdown

# Import IMBD.csv and transform to create the movies dataframe
movies = pd.read_csv("IMDB.csv")

# Rename some columns
movies = movies.rename(columns={
    "primaryTitle" : "movie_title",
    "Description" : "movie_description"
})

# Add the URL using tconts
movies["source"] = "https://www.imdb.com/title/" + movies["tconst"]

# Filter the database
movies = movies.loc[
    # We will only keep those that are actually movies (see column titleType)
    movies["titleType"] == "movie",
    
    # We only want some columns out of the 22 columns we have
    ["movie_title", "movie_description", "source", "genres"]
]

# Allow longer lines to be shown in the .head()
# pd.set_option('display.max_colwidth', None)


# Show the head of movies
print("Shape:", movies.shape)
display('Head:', movies.head())

Shape: (4963, 4)


'Head:'

,movie_title,movie_description,source,genres
0,The Silence of the Lambs,"Jodie Foster stars as Clarice Starling, a top ...",https://www.imdb.com/title/tt0102926,"Crime,Drama,Thriller"
1,Terminator 2: Judgment Day,"In this sequel set eleven years after ""The Ter...",https://www.imdb.com/title/tt0103064,"Action,Sci-Fi"
2,The Lion King,This Disney animated feature follows the adven...,https://www.imdb.com/title/tt0110357,"Adventure,Animation,Drama"
3,Pulp Fiction,Vincent Vega (John Travolta) and Jules Winnfie...,https://www.imdb.com/title/tt0110912,"Crime,Drama"
4,The Shawshank Redemption,Andy Dufresne (Tim Robbins) is sentenced to tw...,https://www.imdb.com/title/tt0111161,Drama


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4963 entries, 0 to 7848
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie_title        4963 non-null   object
 1   movie_description  4822 non-null   object
 2   source             4963 non-null   object
 3   genres             4963 non-null   object
dtypes: object(4)
memory usage: 193.9+ KB


In [5]:
print("Row")

Row


## Task 2: Create Documents from the Data

Later in this project, we will be creating vector embeddings for all of the rows in the `movies` DataFrame. Before we do so, we need to create [Document](https://docs.langchain.com/docs/components/schema/document) objects from the data in the DataFrame. To accomplish this, we can utilize the `DataFrameLoader` class provided by langchain, which allows us to create documents from a pandas DataFrame.

For the main content of the documents, we will create a summary string that includes relevant information about each movie. To achieve this, we will combine the movie title, description, and genre into a `page_content` column. Additionally, we will retain the IMDB link in the `source` column as metadata.

---
In LangChain, "Documents" refer to a structured format for handling text and associated metadata. A `Document` is essentially a piece of text that can be loaded into LangChain along with optional metadata, which is useful for keeping track of information about the document, such as its source.

Here are some key points about `Document`s in LangChain:

1. A `Document` consists of `page_content`, which is a string containing the raw text of the document, and `metadata`, which is a dictionary that stores additional information about the text.

2. LangChain provides document loaders to load data from various sources as `Document`s. These loaders can handle different input formats, such as `.txt` files, CSV files, web page content, and even transcripts of YouTube videos.

3. There are different types of document loaders in LangChain:
   - Transform Loaders: Transform different input formats into the Document format.
   - Public Dataset or Service Loaders: For example, loaders that can retrieve content from public sources like Wikipedia.
   - Proprietary Dataset or Service Loaders: Custom loaders for proprietary sources that may require authentication or special setup.

4. Document loaders can be used for various tasks such as data ingestion, context understanding, creating indexes, and fine-tuning language models. For example, they can break down a large text into smaller chunks to create an index or load structured data from a Pandas DataFrame for fine-tuning.

5. Every document loader exposes two methods:
   - "Load": load documents from the configured source.
   - "Load and split": load documents from the configured source and split them using the passed-in text splitter. They may also implement

"Lazy load": to load documents into memory lazily.

6. Creating a `Document` in LangChain is straightforward. For JavaScript, you can create a `Document` using the `Document` class from the `langchain/document` module. For Python, you import the appropriate loader and use it to load your content into a `Document` object.  


LangChain's approach with `Document`s and document loaders is designed to make it easy to work with large amounts of text data, enabling efficient and flexible data manipulation for applications powered by language models.  

---

### Instructions

- Import `DataFrameLoader` from `langchain.document_loaders`
- Create a column `page_content` that creates strings that contain information about the movie title, genre and description. For example, the first movie should look like this:
```
Title: The Silence of the Lambs
Genre: Crime,Drama,Thriller
Description: Jodie Foster stars as Clarice Starling, a top student at the FBI's training academy. Jack Crawford (Scott Glenn) wants Clarice to interview Dr. Hannibal Lecter (Anthony Hopkins), a brilliant psychiatrist who is also a violent psychopath, serving life behind bars for various acts of murder and cannibalism. Crawford believes that Lecter may have insight into a case and that Starling, as an attractive young woman, may be just the bait to draw him out.
```
- Only keep the columns `page_content` and `source` in the movies DataFrame
- Use `DataFrameLoader` to load documents from the `movies` DataFrame into `docs`. Use `"page_content"` as the `page_content_column`.
- Print the first 3 documents and the total number of documents

In [6]:
# Import DataFrameLoader
from langchain.document_loaders import DataFrameLoader

# Create page content column
movies["page_content"] = f"""Title: {movies["movie_title"]}
Genre: {movies["genres"]}
Description: {movies["movie_description"]}"""

display(movies.head())

,movie_title,movie_description,source,genres,page_content
0,The Silence of the Lambs,"Jodie Foster stars as Clarice Starling, a top ...",https://www.imdb.com/title/tt0102926,"Crime,Drama,Thriller",Title: 0 The Silence of the Lambs\n1 ...
1,Terminator 2: Judgment Day,"In this sequel set eleven years after ""The Ter...",https://www.imdb.com/title/tt0103064,"Action,Sci-Fi",Title: 0 The Silence of the Lambs\n1 ...
2,The Lion King,This Disney animated feature follows the adven...,https://www.imdb.com/title/tt0110357,"Adventure,Animation,Drama",Title: 0 The Silence of the Lambs\n1 ...
3,Pulp Fiction,Vincent Vega (John Travolta) and Jules Winnfie...,https://www.imdb.com/title/tt0110912,"Crime,Drama",Title: 0 The Silence of the Lambs\n1 ...
4,The Shawshank Redemption,Andy Dufresne (Tim Robbins) is sentenced to tw...,https://www.imdb.com/title/tt0111161,Drama,Title: 0 The Silence of the Lambs\n1 ...


Note: Double square brackets `[[ ]]` are used when you want to select multiple columns and the result should be a DataFrame, not a Series.

In [7]:
# Drop all columns except for page_content and source, keep just the columns we need
movies = movies[["page_content", "source"]]

# pd.set_option('display.max_colwidth', None) #to show ful length of the content, just visual purposes
display(movies.head())

,page_content,source
0,Title: 0 The Silence of the Lambs\n1 ...,https://www.imdb.com/title/tt0102926
1,Title: 0 The Silence of the Lambs\n1 ...,https://www.imdb.com/title/tt0103064
2,Title: 0 The Silence of the Lambs\n1 ...,https://www.imdb.com/title/tt0110357
3,Title: 0 The Silence of the Lambs\n1 ...,https://www.imdb.com/title/tt0110912
4,Title: 0 The Silence of the Lambs\n1 ...,https://www.imdb.com/title/tt0111161


Now load the documents from the dataframe into `loader`. The page content column is 'movie_description'.  
See documentation: [Pandas DataFrame in Langchain](https://python.langchain.com/docs/integrations/document_loaders/pandas_dataframe)

`DataFrameLoader` for Pandas takes two arguments:
1. Name of the dataframe. In our case, `movies`
1. `page_content_column` with the name of the column in the dataframe that we want to use as main text. In our case `page_content`.

Then use `.load()` to load the data. The outcome will be a list.

In [8]:
# Load the documents from the dataframe into docs
# The page content column is 'movie_description'
loader = DataFrameLoader(movies, page_content_column="page_content").load()

In [9]:
# Print the first 3 documents and the number of documents
print(type(loader))
print(loader[:2])

<class 'list'>
[Document(page_content='Title: 0         The Silence of the Lambs\n1       Terminator 2: Judgment Day\n2                    The Lion King\n3                     Pulp Fiction\n4         The Shawshank Redemption\n                   ...            \n7843                     Uncle Tom\n7844           Small Engine Repair\n7845                The Blonde One\n7847                     Two of Us\n7848            End of the Century\nName: movie_title, Length: 4963, dtype: object\nGenre: 0            Crime,Drama,Thriller\n1                   Action,Sci-Fi\n2       Adventure,Animation,Drama\n3                     Crime,Drama\n4                           Drama\n                  ...            \n7843          Documentary,History\n7844        Comedy,Drama,Thriller\n7845                Drama,Romance\n7847                Drama,Romance\n7848                        Drama\nName: genres, Length: 4963, dtype: object\nDescription: 0       Jodie Foster stars as Clarice Starling, a top ...\n1  

## Task 3: Estimate the Cost of Embedding

We're going to be using OpenAI to calculate [vector embeddings](https://platform.openai.com/docs/guides/embeddings/embeddings) of the document texts. Creating embeddings is a form of dimensionality reduction, where we assign the text to a point in an N-dimensional space. Texts that are semantically close to each other should end up being close to each other in the N-dimensional space.

Luckily, OpenAI has several models that are trained to calculate these kinds of embeddings, so we don't have to do that ourselves. Of course, a cost is associated with this. You can derive the cost from the [pricing page of OpenAI](https://openai.com/pricing).

The calculation is based on the amount of _tokens_ in the text. All text is encoded into tokens to be used by OpenAI. On average, a token consists of roughly 3 characters. However, we can calculate the exact tokens for a string of text by using the `tiktoken` package.

The goal of this task is to calculate the number of tokens in the documents, to then extrapolate the estimated cost.

### Instructions

- Import `tiktoken`
- Create the encoder, use the `"cl100k_base"` encoder. This is the encoder used by OpenAI to calculate the embeddings for text using the `text-embedding-ada-002` model.
- Create a list that contains the amount of tokens for each document
- Calculate the estimated cost: the sum of all tokens, divided by 1000 tokens, multiplied with $0.0001

In [25]:
# Import tiktoken
import tiktoken

# Create the encoder
encoder = tiktoken.get_encoding("cl100k_base")

docs = loader

token_per_docs = [len(encoder.encode(doc.page_content)) for doc in docs]

total_tokens = len(token_per_docs) * token_per_docs[1] #if all have the same number

total_tokens

1746976

In [23]:
# Total number of tokens (second way)
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

n = 1746976 #this has been changed, must be 0 at the begining
# for doc in docs:
#     n += num_tokens_from_string(doc.page_content, "cl100k_base")
print(n)

1746976


In [26]:
total_price = n * (0.0004 / 1000)

total_price

0.6987904

## Task 4: Create the Index on Pinecone

Looks like calculating the embeddings is not going to be too expensive. It's always smart to get a rough estimate on the amount of tokens used, so you get an idea of the cost of calculating the embeddings using OpenAI.

Now we're ready to create the index on Pinecone. An [index in Pinecone](https://docs.pinecone.io/docs/indexes) can be used to store vectors. You can compare an index in Pinecone to a table in SQL, it stores information of one type of object.

In a later task, we'll be creating vectors from the documents we just created using OpenAI's second-generation embedding model. It's important to already know the embeddings we're going to use since we need to know the output dimensions to create an index. For `text-embedding-ada-002`, this is `1536` dimensions ([source](https://platform.openai.com/docs/guides/embeddings/second-generation-models)).

At the end of this task, you should be able to find your new index, `imdb-movies`, in the [Pinecone UI](https://app.pinecone.io/).

![Pinecone UI](pinecone_ui.png)

### Instructions

- Import `pinecone`
- Use `.init` to initialize the Pinecone client with the `"PINECONE_API_KEY"` environment variable. Use the `"gcp-starter"` environment on Pinecone.
- Print all the indexes on Pinecone by using `.list_indexes` on the client.
- Use `.create_index` to create an index with the name `"imdb-movies"`, but only if it does not exist yet. The metric we'll use is the `"cosine"` distance, and as we mentioned above, the embeddings wil have `1536` dimensions.

In [29]:
# Import pinecone
import pinecone
# Please note that the API KEYs have been imported at the begining of this file

# Initialize pinecone

pinecone.init(
    api_key=PINECONCE_API_KEY,
    environment=PINECONCE_ENVIRONMENT
)

## Vector Similarity

Source: [Pinecone - Vector Similarity Explained](https://www.pinecone.io/learn/vector-similarity/)

We will look at three common vector similarity metrics: Euclidean distance, cosine similarity, and dot product similarity. Understanding the benefits and drawbacks of each metric will enable you to make more informed decisions when deciding on the best similarity metric for your use case. The basic rule of thumb in selecting the best similarity metric for your Pinecone index is to _match it to the one used to train your embedding model_. For example, the all-MiniLM-L6-v2 model was trained using cosine similarity — so using cosine similarity for the index will produce the most accurate result.

#### Similarity metrics:

| Similarity Metric    | Vector properties considered |
|----------------------|------------------------------|
| Euclidean distance   | Magnitudes and direction     |
| Cosine similarity    | Only direction               |
| Dot product similarity | Magnitudes and direction   |

### Euclidean Distance

Euclidean distance is the straight-line distance between two vectors in a multidimensional space. Euclidean distance is a very straightforward similarity metric in that it literally reflects the distance between each of the values of the vectors being compared: if the Euclidean distance is very small, then the values of each coordinate in the vectors are very close. This is not true in general for dot product or cosine. In most cases, you won’t use it with deep learning models but rather with models created with more basic vector encoding methods like LSH (Locality Sensitive Hashing). More generally, Euclidean distance is a natural choice when the model wasn’t trained with a specific loss function.

### Dot Product Similarity

The dot product similarity metric for two vectors is calculated by adding the products of the vectors' corresponding components. The dot product can be affected by the length and direction of the vectors. When two vectors have the same length but different directions, the dot product will be larger if the two vectors are pointing in the same direction and smaller if they are pointing in opposite directions. imagine two vectors represented by arrows, vector a and vector b. If vectors a and b are pointing in the same direction as each other, the dot product of a and b will be larger than if a and b were pointing in opposite directions.

You’re likely to run into many Large Language Models (LLMs) that use dot product for training and as we stated before, the rule of thumb would be to use dot product as the similarity metric for your Pinecone metric. For example, the msmarco-bert-base-dot-v5 model on Hugging Face specifies the “suitable scoring functions” to be only dot product.

### Cosine Similarity

Cosine similarity is a measure of the angle between two vectors. It is computed by taking the dot product of the vectors and dividing it by the product of their magnitudes. This metric is not affected by the size of the vector but only by the angle between them. This means that vectors with large or small values will have the same cosine similarity as long as they point in the same direction.

An example use case for cosine similarity is solving semantic search and document classification problems since it allows you to compare the direction of the vectors (i.e., the overall content of the documents). Similarly, recommendation systems that aim to recommend items to users based on their past behavior could use this similarity metric.

Cosine similarity is probably not suitable when you have data where the magnitude of the vectors is important and should be taken into account when determining similarity. For example, it is not appropriate for comparing the similarity of image embeddings based on pixel intensities.

In [ ]:
# Print the indexes
print(pinecone.list_indexes())

index_name = "imdb-movies"

# First check that the given index does not exist yet
if index_name not in pinecone.list_indexes():
    # Create the 'imbd-movies' index if it does not exist
    pinecone.create_index(
        name=index_name.
        distance = "cosine",     
    )

## Task 5: Fill the Index with the Documents

Now that we have the vector index at our disposal, it's time to populate it with some vectors. In this task, we'll need to:

1. Generate vector embeddings for all documents in `docs`. We'll utilize OpenAI for this purpose. langchain provides a convenient helper for this task, `langchain.embeddings.openai.OpenAIEmbeddings`, which you can use to generate embeddings using the latest `text-embedding-ada-002` model.
2. Populate the vector index in Pinecone with these embeddings. Fortunately, langchain also offers assistance with this through the [`langchain.vectorstores.Pinecone`](https://python.langchain.com/docs/integrations/vectorstores/pinecone) helper.

These two steps can be combined using the convenient helper method `.from_document` of the `Pinecone` class. This method accepts an embedding model as input and efficiently calculates the embeddings, subsequently uploading them to Pinecone. We will also introduce some control flow to the code to ensure we do not add data to the Pinecone index if it already contains data. To achieve this, we can make use of the `.from_existing_index` method of `Pinecone`.

In addition to storing vectors, Pinecone allows the storage of additional metadata. When using the langchain helpers, it automatically assumes that vectors should be created from the `page_content` property of each `Document`. All other properties will be included as metadata.

You can verify that everything has worked correctly by accessing the `imdb-movies` index in the Pinecone UI.

![Pinecone UI showing the imdb-movies index](pinecone_ui_index.png)

### Instructions

- Import `OpenAIEmbeddings` from `langchain.embeddings.openai` and `Pinecone` from `langchain.vectorstores` and `Index` from `pinecone.index`
- Create the `embeddings` object, which should be an instance of `OpenAIEmbeddings`. The defaults are good to go here.
- Use `Pinecone.from_documents` to fill up the vector index on Pinecone using the given documents and embeddings object. This will take a while to run, as it will automatically calculate embeddings from all of the `page_content` properties of the documents, and upload that along with metadata to Pinecone. Assign the result to `docsearch`.
   - Some control flow code is already provided for you, this will make sure you use the existing index if it already contains some vectors and avoids filling it up twice.
- Test out the `docsearch` vector database object, by calling `.as_retriever().get_relevant_documents` with a given question. This will first create a [retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/) from the vector database, and then use that to match the most similar documents in the database.

In [ ]:
# Import OpenAIEmbeddings, Pinecone and Index


# Create the embeddings object


index = Index(index_name)

# Check if there is already some data in the index on Pinecone
if index.describe_index_stats()['total_vector_count'] > 0:
    # If there is, use from_existing_index to use the vector store
    
else:
    # If there is not, use from_documents to fill the vector store
    

question = "What's a good movie about an epic viking?"
    
# Use the vector database as a retriever and get the relevant documents for a quesiton


## Task 6: Create Prompts for RAG

In the previous task, we observed that the vector store can be utilized to retrieve relevant documents related to specific queries. For instance, when asked "What's a good movie about vikings?", the movie 'The Northman' was returned as a result. It is important to note that we did not incorporate any measure of movie quality into the system, so the notion of the movie being "good" is not explicitly encoded in the embeddings. It is crucial to always consider the data provided to the system and interpret the results of the AI system within that context. To enhance the results, one approach could be to include information about the movie quality in the movie description.

The remarkable aspect of RAG is the ability to provide relevant context to the LLM within the prompt itself. In the aforementioned example, we would include a description of 'The Northman' in the prompt, enabling the LLM to generate factual information beyond its knowledge cutoff. 'The Northman' was released in 2022, while the knowledge cutoff for the `gpt-3.5-turbo` model is set at September 2021.

Now that you understand how the retriever can be employed to retrieve relevant documents from the vector database, we need to devise a prompt that presents this information to the LLM when we pose a question.

We require two types of [prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/):
- A template that demonstrates how the information in relevant documents is presented to the LLM
- A template that combines the context with the rest of the prompt

Some example prompt templates are provided in the sample code below, which you are free to edit. Notice that these example templates contain `=========` separators between different parts of the text. These kinds of delimiters are a common tactic to help the LLM distinguish between different parts of your input prompt.

### Instructions

- Import `PromptTemplate` from `langchain.prompts`
- Some example prompt templates are already provided for you. You are free to adapt them at your will. There are two prompt templates:
  - `DOCUMENT_PROMPT`: this template shows how a summary text is created for each document. The properties between the curly brackets (`{`) are replaced with the properties of each `Document`.
  - `QUESTION_PROMPT`: this template creates the full prompt that is sent to the LLM. `question` is replaced by the question asked by the user, and `summaries` is replaced with the summary of each relevant document, created by the `DOCUMENT_PROMPT` template
- Create the `PromptTemplate` objects by using `PromptTemplate.from_template`. Call them `document_prompt` and `question_prompt`, respectively.

In [ ]:
# Import PromptTemplate


# Read/adapt the prompts below at will
DOCUMENT_PROMPT = """{page_content}
IMDB link: {source}
========="""

QUESTION_PROMPT = """Given the following extracted parts of a movie database and a question, create a final answer with the IMDB link as source ("SOURCE").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCE" part in your answer.

QUESTION: What's a good movie about a robot to watch with my kid?
=========
Title: A.I. Artificial Intelligence
Genre: Drama,Sci-Fi
Description: A robotic boy, the first programmed to love, David (Haley Joel Osment) is adopted as a test case by a Cybertronics employee (Sam Robards) and his wife (Frances O'Connor). Though he gradually becomes their child, a series of unexpected circumstances make this life impossible for David. Without final acceptance by humans or machines, David embarks on a journey to discover where he truly belongs, uncovering a world in which the line between robot and machine is both vast and profoundly thin.
IMDB link: https://www.imdb.com/title/tt0212720
=========
Title: I, Robot
Genre: Action,Mystery,Sci-Fi
Description: In 2035, highly intelligent robots fill public service positions throughout the world, operating under three rules to keep humans safe. Despite his dark history with robotics, Detective Del Spooner (Will Smith) investigates the alleged suicide of U.S. Robotics founder Alfred Lanning (James Cromwell) and believes that a human-like robot (Alan Tudyk) murdered him. With the help of a robot expert (Bridget Moynahan), Spooner discovers a conspiracy that may enslave the human race.
IMDB link: https://www.imdb.com/title/tt0343818
=========
Title: The Iron Giant
Genre: Action,Adventure,Animation
Description: In this animated adaptation of Ted Hughes' Cold War fable, a giant alien robot (Vin Diesel) crash-lands near the small town of Rockwell, Maine, in 1957. Exploring the area, a local 9-year-old boy, Hogarth, discovers the robot, and soon forms an unlikely friendship with him. When a paranoid government agent, Kent Mansley, becomes determined to destroy the robot, Hogarth and beatnik Dean McCoppin (Harry Connick Jr.) must do what they can to save the misunderstood machine.
IMDB link: https://www.imdb.com/title/tt0129167
=========
FINAL ANSWER: 'The Iron Giant' is an animated movie about a friendship between a robot and a kid. It would be a good movie to watch with a kid.
SOURCE: https://www.imdb.com/title/tt0129167

QUESTION: {question}
=========
{summaries}
FINAL ANSWER:"""

# Create prompt template objects


## Task 7: Chain Everything Together to Perform RAG

Finally, we have the vector index filled up with information, we have the prompt templates set up. That means we have everything we need to build a question-answering bot, which can use the information retrieved from Pinecone to answer questions about movies.

We'll use the `gpt-3.5-turbo` model of OpenAI in order to provide a completion for the question prompt above.

Langchain provides a convenient concept, called [chains](https://python.langchain.com/docs/modules/chains/), that does some of the heavy lifting when you need to combine multiple AI systems into a single application. For the purpose of this project, we'll be using the `RetrievalQAWithSourcesChain` class. This chain will accept a `question` and a `retriever`. When asked a question, it will first use the retriever to retrieve relevant documents. Afterwards, it will combine the documents into a prompt and send it to the LLM to provide a completion.

### Instructions

- Import `RetrievalQAWithSourcesChain` from `langchain.chains` and `ChatOpenAI` from `langchain.chat_models`
- Use `RetrievalQAWithSourcesChain` to create the chain to answer questions. Use the `.from_chain_type` method:
  - Set `chain_type` set to `"stuff"`. This is the simplest type of chain, and will just stuff the document context in one prompt.
  - Set `llm` to an instance of `ChatOpenAI` with `model_name` set to `"gpt-3.5-turbo"` and `temperature` set to `0`.
  - Use the `PromptTemplate` objects you created above to pass to `chain_type_kwargs`
  - As a retriever, use the `docsearch.as_retriever` method you've seen before

In [ ]:
# Import RetrievalQAWithSourcesChain and ChatOpenAI


# Create the QA bot LLM chain


# Ask the LLM a question about movies


## Task 8: Add Debug Logging

Let's take a moment to address what we've achieved by using RAG, which would be impossible to achieve with just using `gpt-3.5-turbo` as an LLM:

1. We enabled the LLM to answer the question with factual information, which can even be information from after ChatGPT's knowledge cutoff (which is September 2021).
2. We enabled the LLM to provide sources with the answer it generates.

Pretty neat, right?

We saw that langchain is very convenient when it comes to quickly creating smart AI systems. However, for learning, it can be quite challenging to understand what's happening behind the scenes. For example, from the code in Task 7, it's not clear that `qa_with_sources` actually first calls Pinecone to retrieve documents, then uses those documents to fill in the prompt to send along to the `gpt-3.5-turbo` LLM.

Let's look at how we can get some more insights into how this all works.

### Instructions

- Import `langchain`
- Set `.debug` to `True` on `langchain`
- Run `qa_with_sources(question)` again

Observe the information that is printed in the output. Langchain enables you to run chains of LLMs or other AI systems, one after the other. The input for the next chain is passed on from the previous, where new information can be added by, for example, using embeddings to find relevant documents. Each chain or LLM is marked with a tag like `[chain/start]` or `[llm/start]`. When a final response is fetched from the last part of the chain, the output travels back up the chain. This is marked with the `[chain/end]` and `[llm/end]` marks.

In [ ]:
# Import langchain


# Enable debug logging


# Ask the LLM a question about movies
